In [1]:
import pandas as pd
import numpy as np
import spacy
import random
# import es_core_news_sm
from hunspell import Hunspell
import treetaggerwrapper
import regex
from sklearn.model_selection import KFold


random.seed(0)

c:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\DRamos\AppData\Local\Programs\Python\Python38\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
# data import
data = pd.read_excel(r'../data/Datos Codificados Estudio v2a.xlsx')

In [3]:
data_matrix = pd.concat(
    (data.iloc[:,2],    # Cue/Concept
    data.iloc[:,6],     # Codification
    data.iloc[:,3]      # Description of the cue
    ),
    axis = 1
)

In [4]:
data_matrix

,Palabra (concepto),Codificación,Descripción
0,Compasión,sentimiento,sentimiento
1,Compasión,perdón,por lo que se le perdona la vida a alguien en ...
2,plan,objetivos,necesario para cumplir objetivos
3,plan,organización,organizar recursos o personas
4,plan,estrategia,estrategia
...,...,...,...
4933,Obligación,imposición,imposicion
4934,Obligación,extricto,carácter extricto
4935,Obligación,normas,normas
4936,Obligación,deber,deberes


* Corregir Buchanan 
* Sacar las codificaciones a partir de la descripcion en concre
* EJ: descrip: sentimiento  =>  cod: sentimiento

In [4]:
class BuchananProcessor:

    def __init__(self, data):
        self.data = data.copy(deep=True)
        self.nlp = spacy.load('es_core_news_sm')
        self.h = Hunspell('es_CL', hunspell_data_dir='../dict')
        self.tagger = treetaggerwrapper.TreeTagger(TAGLANG='es')

    def _sentenceCheck(self, text):
        """
        Recives a string and returns the same without special caracters and 
        the correct spelling of the string in spanish
        """
        # Remove all special caracters 
        words = [t.orth_ for t in self.nlp(text) if not (t.is_punct or t.orth_==" ")]
        tokens = [t.lower() for t in words]

        # Suggestion for each word bad spelled
        c_spell = [self.h.suggest(t)[0] if (self.h.spell(t)==False and len(self.h.suggest(t))>0) else t for t in tokens]

        # Re-join the words in one string
        return " ".join(c_spell)
        
    def _lemmatize(self,text):
        """
        Recives a string and return a list with the word, pos tagging and the lemma,
        for each word
        """
        return [tag for tag in treetaggerwrapper.make_tags(self.tagger.tag_text(text), exclude_nottags=True)]

    def spellCheck(self):
        """
        Check the spelling for each description.
        """
        for i in range(len(self.data.iloc[:,2])):
            self.data.iloc[i,2] = self._sentenceCheck(self.data.iloc[i,2]) 

    def lemmatization(self):
        """
        modify teh column Descripción to a list of tag for each word
        """
        self.data['Descripción'] = [self._lemmatize(self.data.iloc[i,2]) for i in range(len(self.data.iloc[:,2]))]

    def stopWordRemoval(self):
        """
        Removes al stop word from the column Descripción
        """
        for i in range(len(self.data['Descripción'])):
            self.data['Descripción'][i] = [word for word in self.data['Descripción'][i] if self.nlp(word[2])[0].is_stop==False]
    
    def multiWordSequence(self):
        """
        Find patterns from the 'Descripción' column, to create a new column of multiWordSeq
        patterns:
         * adverb-adjective
         * verb-noun
         * verb-adjective-noun
        """
        patterns = {
            "adv-adj": (r'ADV',r'ADJ'),
            "verb-noun":(r'V\w+', r'^(N(P|C))'),
            "verb-adj-noun":(r'V\w+', r'ADJ',r'^(N(P|C))')
        }
        
        # self.data['multiWordSeq']=np.nan
        for j in range(len(self.data['Descripción'])):
            l = self.data['Descripción'][j]
            temp = [word[2] for word in l]

            if len(l)>1:
                for i in range(len(l)-1):
                    if regex.match(patterns['adv-adj'][0],l[i][1]) and regex.match(patterns['adv-adj'][1],l[i+1][1]):
                        temp.append(" ".join((l[i][2],l[i+1][2])))

                    if regex.match(patterns['verb-noun'][0],l[i][1]) and regex.match(patterns['verb-noun'][1],l[i+1][1]):
                        temp.append(" ".join((l[i][2],l[i+1][2])))


            if len(l)>2:
                for i in range(len(l)-2):
                    if (regex.match(patterns['verb-adj-noun'][0],l[i][1]) and
                        regex.match(patterns['verb-adj-noun'][1],l[i+1][1]) and 
                        regex.match(patterns['verb-adj-noun'][2],l[i+2][1])
                    ):
                        temp.append(" ".join((l[i][2],l[i+1][2],l[i+2][2])))

            self.data['Descripción'][j]=temp
        
    def process(self):
        self.spellCheck()
        self.lemmatization()
        self.stopWordRemoval()
        self.multiWordSequence()

In [5]:
P = BuchananProcessor(data_matrix)
P.process()
P.data

,Palabra (concepto),Codificación,Descripción
0,Compasión,sentimiento,[sentimiento]
1,Compasión,perdón,"[él|le, perdonar, vida, a, alguien, película, ..."
2,plan,objetivos,"[necesario, cumplir, objetivo, cumplir objetivo]"
3,plan,organización,"[organizar, recurso, o, persona, organizar rec..."
4,plan,estrategia,[estrategia]
...,...,...,...
4933,Obligación,imposición,[imposición]
4934,Obligación,extricto,"[carácter, extracto]"
4935,Obligación,normas,[norma]
4936,Obligación,deber,[deber]


In [7]:
buchanan_acc = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    test = P.data.iloc[test_index,:]
    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        if len(test.iloc[i,2])>0:
            counter += test.iloc[i,1] == np.random.choice(test.iloc[i,2])
    buchanan_acc[j]=(counter/len_test)
    j+=1

In [8]:
buchanan_acc.mean()

0.3234145100886422

In [9]:
buchanan_acc.std()

0.009905623637539305

In [6]:
buchanan_acc_top3 = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    test = P.data.iloc[test_index,:]
    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        if len(test.iloc[i,2])>0:
            for _ in range(3):
                val = test.iloc[i,1] == np.random.choice(test.iloc[i,2])
                if val:
                    counter += val
                    break
    buchanan_acc_top3[j]=(counter/len_test)
    j+=1

In [7]:
print(buchanan_acc_top3.mean())
print(buchanan_acc_top3.std())

0.4064467633896525
0.015453529934383366


In [8]:
buchanan_acc_top5 = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    test = P.data.iloc[test_index,:]
    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        if len(test.iloc[i,2])>0:
            for _ in range(5):
                val = test.iloc[i,1] == np.random.choice(test.iloc[i,2])
                if val:
                    counter += val
                    break
    buchanan_acc_top5[j]=(counter/len_test)
    j+=1

print(buchanan_acc_top5.mean())
print(buchanan_acc_top5.std())

0.44269450221297924
0.012490722666532762


In [9]:
buchanan_acc_top10 = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    test = P.data.iloc[test_index,:]
    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        if len(test.iloc[i,2])>0:
            for _ in range(10):
                val = test.iloc[i,1] == np.random.choice(test.iloc[i,2])
                if val:
                    counter += val
                    break
    buchanan_acc_top10[j]=(counter/len_test)
    j+=1

print(buchanan_acc_top10.mean())
print(buchanan_acc_top10.std())

0.4722631045699354
0.015738758801024773


In [10]:
buchanan_acc_top50 = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    test = P.data.iloc[test_index,:]
    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        if len(test.iloc[i,2])>0:
            for _ in range(50):
                val = test.iloc[i,1] == np.random.choice(test.iloc[i,2])
                if val:
                    counter += val
                    break
    buchanan_acc_top50[j]=(counter/len_test)
    j+=1

print(buchanan_acc_top50.mean())
print(buchanan_acc_top50.std())

0.4825912982127988
0.0164724344734776


In [10]:
n1 = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    test = P.data.iloc[test_index,:]
    len_test = len(test)
    counter = 0
    
    for i in range(len_test):
        if len(test.iloc[i,2])==1:
            counter += test.iloc[i,1] == np.random.choice(test.iloc[i,2])
    n1[j]=(counter/len_test)
    j+=1

In [11]:
n1.mean()

0.23916973284274518

In [12]:
n1.std()

0.008255645611488232

In [13]:
equal_acc = np.zeros(5)
j = 0
kfold=KFold(n_splits=5)
for train_index, test_index in kfold.split(data_matrix):
    original_test = data_matrix.iloc[test_index,:]
    len_test = len(original_test)
    counter = 0
    
    for i in range(len_test):
        if len(original_test.iloc[i,2])>0:
            counter += original_test.iloc[i,1] == original_test.iloc[i,2]
    equal_acc[j]=(counter/len_test)
    j+=1

In [14]:
equal_acc.mean()

0.14297302175241705

In [15]:
equal_acc.std()

0.020108071814554155